This Notebook evaluates the study comparing the different variants of numerical fluxes for a Mach 4 supersonic flow over a blunt body presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024) 

In [ ]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [ ]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_FluxStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy'.


{ Session Count = 45; Grid Count = 45; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy }

In [24]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-12 14:21:59Z

In [25]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx1-bFlx0-Fphi0	3/12/2024 2:22:42 PM	2b797256...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/12/2024 2:21:40 PM	849b4d77...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	3/12/2024 2:28:00 PM	f0bd6ce4...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx1-bFlx0-Fphi0	3/12/2024 2:24:47 PM	2479e5cf...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/12/2024 2:23:43 PM	4aae3479...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx2-wFLx1-bFlx0-Fphi0	3/12/2024 2:26:53 PM	4ad06c9f...
#6: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx1-bFlx0-Fphi0	3/12/2024 2:29:02 PM	2ed6800e...
#7: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0	3/12/2024 2:25:50 PM	1f9aa518...
#8: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx1-bFlx2-Fphi0*	3/12/2024 3:52:11 PM	74b3f494...
#9: XDGBowShock_TwoLs	XDGBS-p3-10x32

In [26]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

2b797256-e948-4757-8d89-34002943d268

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [27]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [28]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [29]:
var ses=sessions.Where(s => (s.SuccessfulTermination &&s.Name.Contains("bFlx0")&&s.Name.Contains("wFLx0"))); // the one plotted in the publication
ses


#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/12/2024 2:21:40 PM	849b4d77...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	3/12/2024 2:28:00 PM	f0bd6ce4...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/12/2024 2:23:43 PM	4aae3479...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0	3/12/2024 2:25:50 PM	1f9aa518...


# A Helper Function to obtain different plot formats

In [30]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [31]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M511.2,57.1 L564.6,57.1 M45.6,366.2 L51.7,359.5 L57.9,361.3 L64.0,361.5 L70.1,361.6 L76.3,361.8
 L82.4,364.3 L88.6,365.3 L94.7,364.3 L100.8,364.8 L107.0,365.3 L113.1,366.3 L119.2,366.4 L125.4,366.4
 L131.5,366.5 L137.6,366.4 L143.8,366.3 L149.9,366.3 L156.1,366.4 L162.2,366.4 L168.3,366.4 L174.5,366.3
 L180.6,366.3 L186.7,366.4 L192.9,366.4 L199.0,366.4 L205.1,366.3 L211.3,366.4 L217.4,366.4 L223.5,366.4
 L229.7,313.8 L235.8,366.4 L242.0,366.4 L248.1,366.4 L254.2,366.4 L260.4,366.4 L266.5,363.1 L272.6,360.3
 L278.8,363.8 L284.9,366.0 L291.0,374.6 L297.2,416.7 L303.3,439.2 L309.5,437.0 L315.6,439.3 L321.7,440.4
 L327.9,440.9 L334.0,441.0 L340.1,440.5 L346.3,440.7 L352.4,440.5 L358.5,440.6 L364.7,440.8 L370.8,440.8
 L377.0,440.7 L383.1,441.0 L389.2,440.5 L395.4,440.9 L401.5,440.7 L407.6,441.0 L413.8,440.6 L419.9,440.8
 L426.0,440.7 L432.2,440.8 L438.3,440.9 L444.4,440.9 L450.6,476.9 L456.7,478.8 L462.9,480.4 L469.0,480.2
 L475.1,480.0 L481.3,479.7 L487.4,479.7 L493.5,479.9 L499.7,479.7 L505.8,479.8 L511.9,480.1 L518.1,480.2
 L524.2,480.3 L530.4,480.4 L536.5,480.4 L542.6,480.3 L548.8,480.4 L554.9,480.4 L561.0,480.4 L567.2,480.4
 L573.3,480.4 L579.4,480.4 L585.6,480.4 L591.7,480.4 L597.9,480.4 L604.0,480.4 L610.1,480.4 L616.3,480.4
 L622.4,480.4 L628.5,480.4 L634.7,480.4 L640.8,480.4 L646.9,480.4 L653.1,480.4 L659.2,480.4 L665.3,480.4
 L671.5,480.4 L677.6,480.4 L683.8,480.4 L689.9,480.4 L696.0,480.4 L702.2,480.4 L708.3,480.4 L714.4,480.4
 L720.6,480.4 L726.7,480.4 L732.8,480.4 L739.0,480.4 L745.1,480.4 L751.3,480.4 L757.4,480.4 L763.5,480.4
 L769.7,480.4 L775.8,480.4 L781.9,480.4 L788.1,480.4 L794.2,480.4 L800.3,480.4 L806.5,480.4 L812.6,511.2
 L818.8,512.1 L824.9,512.1 L831.0,512.1 L837.2,512.1 L843.3,512.1 L849.4,512.1 L855.6,512.0 L861.7,512.0
 L867.8,512.0 L874.0,512.0 L880.1,512.0 L886.2,512.0 L892.4,512.0 L898.5,512.0 L904.7,512.0 L910.8,512.0
 L916.9,512.0 L923.1,512.0 L929.2,512.0 L935.3,512.0 L941.5,512.0 L947.6,512.0 L953.7,512.0 L959.9,512.0
 L966.0,512.0 L972.2,512.0 L978.3,512.0 L984.4,512.0 L990.6,512.0 L996.7,512.0 L1002.8,512.0 L1009.0,512.0
 L1015.1,512.0 L1021.2,512.0 L1027.4,512.0 L1033.5,512.0 L1039.7,512.0 L1045.8,512.0 L1051.9,512.0 L1058.1,512.0
 L1064.2,512.0 L1070.3,512.0 L1076.5,512.0 L1082.6,512.0 L1088.7,512.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M511.2,81.1 L564.6,81.1 M45.6,366.2 L51.7,359.5 L57.9,361.2 L64.0,361.2 L70.1,361.2 L76.3,361.2
 L82.4,361.2 L88.6,361.2 L94.7,361.2 L100.8,361.2 L107.0,361.2 L113.1,361.2 L119.2,361.3 L125.4,361.3
 L131.5,361.3 L137.6,361.3 L143.8,361.3 L149.9,361.3 L156.1,361.3 L162.2,361.3 L16

## Residual History

In [32]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    lts.EnResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M22.4,57.1 L75.8,57.1 M45.6,105.7 L49.5,110.9 L53.4,113.4 L57.3,113.5 L61.3,113.8 L65.2,113.5
 L69.1,119.5 L73.0,125.7 L76.9,128.7 L80.8,132.4 L84.7,132.6 L88.7,138.3 L92.6,151.7 L96.5,156.7
 L100.4,160.0 L104.3,165.0 L108.2,161.5 L112.1,164.1 L116.0,168.6 L120.0,159.6 L123.9,163.3 L127.8,174.2
 L131.7,178.4 L135.6,163.5 L139.5,170.5 L143.4,165.2 L147.4,170.3 L151.3,172.8 L155.2,182.5 L159.1,159.8
 L163.0,163.9 L166.9,173.6 L170.8,161.0 L174.8,167.0 L178.7,160.1 L182.6,172.1 L186.5,112.7 L190.4,116.2
 L194.3,103.3 L198.2,126.4 L202.2,126.9 L206.1,140.3 L210.0,143.0 L213.9,165.2 L217.8,182.9 L221.7,184.7
 L225.6,185.3 L229.6,188.3 L233.5,187.3 L237.4,194.6 L241.3,203.2 L245.2,182.9 L249.1,190.7 L253.0,192.8
 L256.9,198.0 L260.9,188.9 L264.8,189.9 L268.7,197.4 L272.6,192.0 L276.5,194.7 L280.4,199.9 L284.3,199.3
 L288.3,194.7 L292.2,201.4 L296.1,186.7 L300.0,191.7 L303.9,162.9 L307.8,165.1 L311.7,202.1 L315.7,209.3
 L319.6,210.1 L323.5,205.2 L327.4,214.3 L331.3,219.0 L335.2,217.6 L339.1,220.4 L343.1,222.4 L347.0,223.7
 L350.9,223.1 L354.8,225.8 L358.7,226.2 L362.6,225.8 L366.5,224.7 L370.5,228.0 L374.4,230.2 L378.3,230.8
 L382.2,229.9 L386.1,230.8 L390.0,231.8 L393.9,230.0 L397.8,231.3 L401.8,232.4 L405.7,233.2 L409.6,234.2
 L413.5,235.5 L417.4,235.9 L421.3,236.2 L425.2,236.2 L429.2,236.4 L433.1,237.2 L437.0,237.5 L440.9,237.5
 L444.8,237.9 L448.7,237.8 L452.6,237.9 L456.6,238.0 L460.5,237.9 L464.4,238.2 L468.3,238.8 L472.2,238.9
 L476.1,239.7 L480.0,239.7 L484.0,239.9 L487.9,240.0 L491.8,239.9 L495.7,240.0 L499.6,240.1 L503.5,240.1
 L507.4,240.2 L511.4,240.1 L515.3,240.2 L519.2,240.2 L523.1,240.2 L527.0,240.2 L530.9,240.2 L534.8,200.4
 L538.7,223.0 L542.7,226.1 L546.6,230.2 L550.5,235.4 L554.4,238.5 L558.3,239.7 L562.2,242.6 L566.1,241.8
 L570.1,243.6 L574.0,250.0 L577.9,246.8 L581.8,248.6 L585.7,249.5 L589.6,249.9 L593.5,252.1 L597.5,251.5
 L601.4,253.3 L605.3,254.2 L609.2,253.8 L613.1,254.9 L617.0,255.4 L620.9,256.2 L624.9,256.5 L628.8,256.7
 L632.7,256.8 L636.6,256.8 L640.5,257.1 L644.4,257.0 L648.3,257.4 L652.3,257.6 L656.2,257.6 L660.1,257.8
 L664.0,258.0 L667.9,258.1 L671.8,258.1 L675.7,258.1 L679.6,258.1 L683.6,258.2 L687.5,258.2 L691.4,258.2
 L695.3,258.3 L699.2,258.3 L703.1,258.3 L707.0,258.3 L711.0,258.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M22.4,81.1 L75.8,81.1 M45.6,89.1 L49.5,91.9 L53.4,93.3 L57.3,93.5 L61.3,93.7 L65.2,93.7
 L69.1,98.3 L73.0,100.2 L76.9,104.7 L80.8,106.2 L84.7,107.3 L88.7,109.3 L92.6,111.4 L96.5,111.7
 L100.4,111.6 L104.3,111.7 L108.2,111.8 L112.1,111.9 L116.0,111.

### Plot ShadowFields

In [ ]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm;
